# Get alpha code information
LAST UPDATED: 2025-03-17

Uses the "2024" version of taxonomies from the Institute for Bird Populations (IBP) and the Bird Banding Laboratory (BBL).

Creates: 
* `ibp-alpha-codes_2024.csv`
* `bbl-alpha-codes_2024.csv`

In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## Download code tables
### IBP codes
* The code below reads the CSV from the IBP website
* To capture the correct version, change the last two digits of the year in the filenames below below to the year of the most recent IBP code update (i.e. `IBP-AOS-LIST24.csv`, `ibp-alpha-codes_2024.csv`)
* The following code can be run in a terminal or by using the `%%bash` cell magic in Jupyter Notebook

In [16]:
%%bash
# IBP zipped alpha codes in taxonomic order
wget https://www.birdpop.org/docs/misc/IBPAOU.zip
unzip IBPAOU.zip
mv IBP-AOS-LIST24.csv ibp-alpha-codes_2024.csv
rm IBPAOU.zip

--2025-03-17 17:29:21--  https://www.birdpop.org/docs/misc/IBPAOU.zip
Resolving www.birdpop.org (www.birdpop.org)... 67.227.214.192
Connecting to www.birdpop.org (www.birdpop.org)|67.227.214.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49035 (48K) [application/zip]
Saving to: ‘IBPAOU.zip’

     0K .......... .......... .......... .......... .......   100%  734K=0.07s

2025-03-17 17:29:22 (734 KB/s) - ‘IBPAOU.zip’ saved [49035/49035]



Archive:  IBPAOU.zip
  inflating: IBP-AOS-LIST24.csv      


In [17]:
ibp_table.head()

,non_species,true_alpha,conflict,common_name,scientific_name,true_alpha_6,conflict_6,expected_alpha
0,NaN,HITI,NaN,Highland Tinamou,Nothocercus bonapartei,NOTBON,NaN,HITI
1,NaN,GRTI,NaN,Great Tinamou,Tinamus major,TINMAJ,NaN,GRTI
2,NaN,LITI,NaN,Little Tinamou,Crypturellus soui,CRYSOU,NaN,LITI
3,NaN,THTI,NaN,Thicket Tinamou,Crypturellus cinnamomeus,CRYCIN,NaN,THTI
4,NaN,SBTI,NaN,Slaty-breasted Tinamou,Crypturellus boucardi,CRYBOU,NaN,SBTI


In [18]:
ibp_table = pd.read_csv("ibp-alpha-codes_2024.csv")
ibp_table.drop(["B4", "B1", "B2"], axis=1, inplace=True)
ibp_table.columns = ['non_species', 'true_alpha', 'conflict', 'common_name', 'scientific_name', 'true_alpha_6', 'conflict_6']
ibp_table.to_csv("ibp-alpha-codes_2024.csv", index=False)

### BBL codes
Bird Banding Laboratory list: 
* The code below grabs the table from this website - https://www.pwrc.usgs.gov/BBL/Bander_Portal/login/speclist.php 
* This requires installation of `pandas` and `lxml`

In [19]:
[bbl_table] = pd.read_html("https://www.pwrc.usgs.gov/BBL/Bander_Portal/login/speclist.php")

In [20]:
bbl_table.head()

,Species Number,Alpha Code,Common Name,Band Size,French Name,Scientific Name,T & E*,Comments,Taxonomic Order
0,10,WEGR,Western Grebe,"7B, 7, 7A",Grèbe élégant,Aechmophorus occidentalis,NaN,NaN,100
1,11,CLGR,Clark's Grebe,"7A, 7, 7B",Grèbe à face blanche,Aechmophorus clarkii,NaN,NaN,102
2,12,WCGR,Western/Clark's Grebe,"7A, 7B",Grèbe à face blanche/Grèbe élégant non identifiée,NaN,NaN,Unspecified Aechmophorus occidentalis or A. cl...,101
3,20,RNGR,Red-necked Grebe,"7A, 7, 7B",Grèbe jougris,Podiceps grisegena,NaN,NaN,98
4,30,HOGR,Horned Grebe,"6, 5, 5A",Grèbe esclavon,Podiceps auritus,Québec,Magdalen Islands population is endangered. | L...,97


In [21]:
bbl_table = bbl_table.drop(['Species Number', 'Band Size', 'French Name', 'Taxonomic Order'], axis=1)
bbl_table.columns = ['true_alpha', 'common_name', 'scientific_name', 't_and_e', 'comments']

Change the year below.

In [22]:
bbl_table.to_csv("bbl-alpha-codes_2024.csv", index=False)

## Get expected alpha codes

Change the years below.

In [23]:
bbl_table = pd.read_csv("bbl-alpha-codes_2024.csv")
ibp_table = pd.read_csv("ibp-alpha-codes_2024.csv")

Produce expected alpha codes from the common names. Rules adapted from: https://sora.unm.edu/sites/default/files/journals/nabb/v028n02/p0064-p0079.pdf

Mostly works. Doesn't work for:

Poo-uli: 
* Desired output: POUL
* Actual output: POO-

Chuck-will's-Widow:
* Desired output: CWWI
* Actual output: CHUC

In [24]:
def expected_alpha(common_name):
    split_spaces = common_name.split(' ')
    split_spaces_hyphens = common_name.replace('-', ' ').replace('/', ' ').split(' ')
    
    # Canvasback --> CANV
    # Also: Chuck-will's-Widow --> CHUC, an error :(
    if len(split_spaces) == 1:
        return split_spaces[0][:4]
    
    if len(split_spaces) == 2:
        
        # Eastern Towhee --> EATO
        if len(split_spaces_hyphens) == 2:
            return split_spaces[0][:2] + split_spaces[1][:2]
    
        elif len(split_spaces_hyphens) == 3:
            
            # Yellow-rumped Warbler --> YRWA
            if ('-' in split_spaces[0]) or ('/' in split_spaces[0]):
                return split_spaces_hyphens[0][0] + split_spaces_hyphens[1][0] + split_spaces_hyphens[2][:2]
            
            # Eastern Screech-Owl --> EASO
            else:
                return split_spaces_hyphens[0][:2] + split_spaces_hyphens[1][0] + split_spaces_hyphens[2][0]
        
        # Black-and-white Warbler --> BAWW
        # Band-rumped Storm-Petrel --> BRSP [this produces a conflict!]
        else:
            return split_spaces_hyphens[0][0] + split_spaces_hyphens[1][0] + split_spaces_hyphens[2][0] + split_spaces_hyphens[3][0]
        
    elif len(split_spaces) == 3:
        # American Tree Sparrow --> ATSP
        if ('-' not in common_name) and ('/' not in common_name):
            return split_spaces[0][0] + split_spaces[1][0] + split_spaces[2][:2]
        
        # Great Black-backed Gull --> GBBG
        else:
            return split_spaces_hyphens[0][0] + split_spaces_hyphens[1][0] + split_spaces_hyphens[2][0] + split_spaces_hyphens[3][0]
    
    # Puget Sound White-crowned Sparrow --> PSWP
    else:
            return split_spaces[0][0] + split_spaces[1][0] + split_spaces[2][0] + split_spaces[3][0]
        


In [25]:
# Put expected codes into IBP table
ibp_table['expected_alpha'] = ibp_table['common_name'].apply(expected_alpha).str.upper()
ibp_table[ibp_table['expected_alpha'] != ibp_table['true_alpha']].head()

,non_species,true_alpha,conflict,common_name,scientific_name,true_alpha_6,conflict_6,expected_alpha
18,+,SRGH,NaN,Snow X Ross's Goose Hybrid,Anser caerulescens x rossii,ANSCAR,NaN,SXRG
23,+,TWFG,NaN,Tule Greater White-fronted Goose,Anser albifrons elgasi,ANSALE,NaN,TGWG
30,+,BABI,NaN,Black X Atlantic Brant Intergrade,Branta bernicula bernicula x nigrican,BRABBN,NaN,BXAB
32,NaN,BARG,*,Barnacle Goose,Branta leucopsis,BRALEU,NaN,BAGO
33,NaN,CACG,*,Cackling Goose,Branta hutchinsii,BRAHUT,NaN,CAGO


In [26]:
# Put expected codes into BBL data
bbl_table['expected_alpha'] = bbl_table['common_name'].apply(expected_alpha).str.upper()
bbl_table[bbl_table['expected_alpha'] != bbl_table['true_alpha']].head()

,true_alpha,common_name,scientific_name,t_and_e,comments,expected_alpha
56,HERG,Herring Gull,Larus argentatus,NaN,NaN,HEGU
64,SHOG,Short-billed Gull,Larus brachyrhynchus,NaN,"from Mew Gull Split, AOS 62nd Supplement (2021)",SBGU
65,HEEG,Heermann's Gull,Larus heermanni,NaN,NaN,HEGU
74,ROYT,Royal Tern,Thalasseus maximus,NaN,NaN,ROTE
78,CAYT,Cayenne Tern,Thalasseus acuflavidus eurygnathus,NaN,NaN,CATE


Sanity check: the dataframe where the expected does not equal the true should either have a star in the `'CONF'` column or a plus in the `'SP'` column.

What species is this NOT true for?

In [27]:
# Expected != True
unmatched = ibp_table[ibp_table['expected_alpha'] != ibp_table['true_alpha']]

# If expected != true and there's no "*" in the conflict column, print the row
for idx, row in unmatched.iterrows():
    try:
        assert (row['non_species'] == '+') or (row['conflict'] == '*')
    except AssertionError:
        print(row)

non_species                          NaN
true_alpha                          SPBO
conflict                             NaN
common_name        Spot-bellied Bobwhite
scientific_name       Colinus leucopogon
true_alpha_6                      COLLEU
conflict_6                           NaN
expected_alpha                      SBBO
Name: 130, dtype: object
non_species                             NaN
true_alpha                             CWWI
conflict                                NaN
common_name              Chuck-will's-widow
scientific_name    Antrostomus carolinensis
true_alpha_6                         ANTCAR
conflict_6                              NaN
expected_alpha                         CHUC
Name: 270, dtype: object
non_species                          NaN
true_alpha                          RTHE
conflict                             NaN
common_name        Rufescent Tiger-Heron
scientific_name       Tigrisoma lineatum
true_alpha_6                      TIGLIN
conflict_6              

Most of these are understandable. 

Failures of the alpha code estimation algorithm:
* Spot-bellied Bobwhite (SPBO): Unclear why the code is different from expected.
* CWWI and Poo-uli: hyphen problems
* Rufescent Tiger-Heron and Fasciated Tiger-Heron: whoever made these alpha codes disregarded the fact that Tiger-Heron is hyphenated
* LeConte's Thrasher, LeConte's Sparrow, McKay's Bunting, and MacGillivray's Warbler: apparently the "mc" and "le" patronymics have a different naming scheme.

Species that should be listed as having conflicts but don't:
* Eurasian Goshawk (EGOS), Puerto Rican Owl (PRIO): Should have a conflict * (with European Goldfinch, Prothonotary Warbler) - but does not
* Olivaceous Flatbill, Ochre-lored Flatbill: Should have a conflict * (with each other) - but do not
* Jamaican Pewee (JAPE): Should have a conflict * (with Jamaican Petrel) - but does not
* Rufous-backed Wren (RBAW): Should have a conflict * (with Rufous-backed Wren) - but does not
* Chestnut-capped Warbler (CCAW): Should have a conflict * (with Crescent-chested Warbler) - but does not

## Create "display" common names / comparison names

These will be fixed up in a "display name" column

### Fix display names

* How hybrids are written
* How slashes and unidentified birds are written


In [32]:
bbl_table['display_name'] = bbl_table['common_name'].str.replace(' X ', ' x ')

In [33]:
ibp_table['display_name'] = ibp_table['common_name'].str.replace(' X ', ' x ')

### What are the discrepancies between BBL and IBP common names?

Potential discrepancies:
* Hyphenation differences
* Capitalization differences
* European vs Eurasian GWT; American GWT


First, look through all of IBP and print the codes where BBL name is not equal to IBP name:

In [52]:
ibp_tc = ibp_table[['true_alpha', 'display_name', 'scientific_name', 'common_name']]
bbl_tc = bbl_table[['true_alpha', 'display_name', 'scientific_name', 'common_name']]

# Look through all the IBP species
ibp_idxes_checked = []
for idx, row in ibp_tc.iterrows():
    
    # Record which IBP entries we've looked through
    ibp_idxes_checked.append(idx)
    
    # Find the BBL name for this IBP species
    bbl_for_this = bbl_tc[bbl_tc['true_alpha'] == row['true_alpha']]
    
    # If BBL has a code for this species and the code is not equal to the IBP code
    if len(bbl_for_this) > 0 and bbl_for_this['display_name'].values[0] != row['display_name']: 
        print('IBP SPECIES:', " -- ".join(row))
        print('BBL SPECIES:', " -- ".join([str(x) for x in bbl_for_this.values[0]]))
        print()
        print()

IBP SPECIES: TWFG -- Tule Greater White-fronted Goose -- Anser albifrons elgasi -- Tule Greater White-fronted Goose
BBL SPECIES: TWFG -- Tule White-fronted Goose -- Anser albifrons elgasi -- Tule White-fronted Goose


IBP SPECIES: BARG -- Barnacle Goose -- Branta leucopsis -- Barnacle Goose
BBL SPECIES: BARG -- Bar-tailed Godwit -- Limosa lapponica -- Bar-tailed Godwit


IBP SPECIES: SCGO -- Small Canada Goose -- Branta canadensis parvipes -- Small Canada Goose
BBL SPECIES: SCGO -- Small Canada/Cackling Goose -- Branta canadensis parvipes -- Small Canada/Cackling Goose


IBP SPECIES: AGWT -- American Green-winged Teal -- Anas crecca carolinensis -- American Green-winged Teal
BBL SPECIES: AGWT -- Green-winged Teal -- Anas crecca -- Green-winged Teal


IBP SPECIES: EGWT -- Eurasian Green-winged Teal -- Anas crecca crecca -- Eurasian Green-winged Teal
BBL SPECIES: EGWT -- European Green-winged Teal -- Anas crecca crecca -- European Green-winged Teal


IBP SPECIES: BLAG -- Black Guan -- Ch

Now, check through BBL as well, skipping any that already came up above also need to be fixed...

In [53]:
for idx, row in bbl_tc.iterrows():
    ibp_for_this = ibp_tc[ibp_tc['true_alpha'] == row['true_alpha']]
    if (len(ibp_for_this) > 0) and (ibp_for_this['common_name'].values[0] != row['common_name']) and (ibp_for_this.index[0] not in ibp_idxes_checked):
        print('BBL SPECIES:', row)
        print('IBP SPECIES:', ibp_for_this)
        print()
        print()
        print()

Okay, we got them all in the previous round.

Now, check through the remaining IBP entries for ones that there's no BBL match for. Which ones have expected alpha code != true?

In [50]:
ibp_tc = ibp_table
bbl_tc = bbl_table

for idx, row in ibp_tc.iterrows():
    bbl_for_this = bbl_tc[bbl_tc['true_alpha'] == row['true_alpha']]
    if len(bbl_for_this) == 0 and row['expected_alpha'] != row['true_alpha']:
        print('IBP SPECIES:', row)
        print()
        print('BBL SPECIES:', bbl_for_this)
        print()
        print()
        print()

IBP SPECIES: non_species                      NaN
true_alpha                      CACG
conflict                           *
common_name           Cackling Goose
scientific_name    Branta hutchinsii
true_alpha_6                  BRAHUT
conflict_6                       NaN
expected_alpha                  CAGO
display_name          Cackling Goose
Name: 33, dtype: object

BBL SPECIES: Empty DataFrame
Columns: [true_alpha, common_name, scientific_name, t_and_e, comments, expected_alpha, display_name]
Index: []



IBP SPECIES: non_species                      NaN
true_alpha                      CANG
conflict                           *
common_name             Canada Goose
scientific_name    Branta canadensis
true_alpha_6                  BRACAN
conflict_6                       NaN
expected_alpha                  CAGO
display_name            Canada Goose
Name: 37, dtype: object

BBL SPECIES: Empty DataFrame
Columns: [true_alpha, common_name, scientific_name, t_and_e, comments, expected_alpha,

### Unify IBP & BBL common names where they do represent the same taxon

Above, I noticed that some of the entries that really represent the same taxon were written differently in the two tables. 

Change the display names in one of the tables to match the other table to resolve these discrepancies. Common name to display name changes:

```
BBL 'Tule White-fronted Goose' --> 'Tule Greater White-fronted Goose'
BBL 'Green-winged Teal' --> 'American Green-winged Teal'
BBL 'European Green-winged Teal' --> 'Eurasian Green-winged Teal'
BBL 'Western Cattle Egret' --> 'Western Cattle-Egret'
IBP 'Gray-cheeked/Bicknell's Thrush' --> 'Unidentified Gray-cheeked/Bicknell's Thrush'
IBP 'Sharp-tailed Sparrow' --> 'Unidentified Sharp-tailed Sparrow'
IBP 'Bullock's x Baltimore Oriole Hybrid' --> 'Baltimore x Bullock's Oriole Hybrid
IBP 'Dusky Seaside-Sparrow' --> 'Dusky Seaside Sparrow'
IBP 'Cape Sable Seaside-Sparrow' --> 'Cape Sable Seaside Sparrow'
IBP 'Blue-wg. x Golden-wg. Warbler Hybrid' --> 'Golden-winged x Blue-winged Warbler Hybrid'
```

TODO: Should we add "GWTE"? DEJU? Etc.

In [67]:
###
# BBL 'Tule White-fronted Goose' --> 'Tule Greater White-fronted Goose'
# BBL 'Green-winged Teal' --> 'American Green-winged Teal'
# BBL 'European Green-winged Teal' --> 'Eurasian Green-winged Teal'
# BBL 'Western Cattle Egret' --> 'Western Cattle-Egret'
# IBP 'Gray-cheeked/Bicknell's Thrush' --> 'Unidentified Gray-cheeked/Bicknell's Thrush'
# IBP 'Sharp-tailed Sparrow' --> 'Unidentified Sharp-tailed Sparrow'
# IBP 'Bullock's x Baltimore Oriole Hybrid' --> 'Baltimore x Bullock's Oriole Hybrid'
# IBP 'Dusky Seaside-Sparrow' --> 'Dusky Seaside Sparrow'
# IBP 'Cape Sable Seaside-Sparrow' --> 'Cape Sable Seaside Sparrow'
# IBP 'Blue-wg. x Golden-wg. Warbler Hybrid' --> 'Golden-winged x Blue-winged Warbler Hybrid'

bbl_table.loc[bbl_table[bbl_table['common_name'] == 'Tule White-fronted Goose'].index, 'display_name'] = 'Tule Greater White-fronted Goose'
bbl_table.loc[bbl_table[bbl_table['common_name'] == 'Green-winged Teal'].index, 'display_name'] = 'American Green-winged Teal'
bbl_table.loc[bbl_table[bbl_table['common_name'] == 'European Green-winged Teal'].index, 'display_name'] = 'Eurasian Green-winged Teal'
bbl_table.loc[bbl_table[bbl_table['common_name'] == 'Western Cattle Egret'].index, 'display_name'] = 'Western Cattle-Egret'
ibp_table.loc[ibp_table[ibp_table['common_name'] == "Gray-cheeked/Bicknell's Thrush"].index, 'display_name'] = "Unidentified Gray-cheeked/Bicknell's Thrush"
ibp_table.loc[ibp_table[ibp_table['common_name'] == 'Sharp-tailed Sparrow'].index, 'display_name'] = 'Unidentified Sharp-tailed Sparrow'
ibp_table.loc[ibp_table[ibp_table['common_name'] == "Bullock's x Baltimore Oriole Hybrid"].index, 'display_name'] = "Baltimore x Bullock's Oriole Hybrid"
ibp_table.loc[ibp_table[ibp_table['common_name'] == 'Dusky Seaside-Sparrow'].index, 'display_name'] = 'Dusky Seaside Sparrow'
ibp_table.loc[ibp_table[ibp_table['common_name'] == 'Cape Sable Seaside-Sparrow'].index, 'display_name'] = 'Cape Sable Seaside Sparrow'
ibp_table.loc[ibp_table[ibp_table['common_name'] == 'Blue-wg. x Golden-wg. Warbler Hybrid'].index, 'display_name'] = 'Golden-winged x Blue-winged Warbler Hybrid'


As an example, check to make sure that the old name was successfully changed:
1. The display_name was changed - the new one shows up, and the old one doesn't exist anymore

In [68]:
bbl_table[bbl_table['display_name'] == 'American Green-winged Teal']

,true_alpha,common_name,scientific_name,t_and_e,comments,expected_alpha,display_name
200,AGWT,Green-winged Teal,Anas crecca,NaN,NaN,GWTE,American Green-winged Teal


In [69]:
bbl_table[bbl_table['display_name'] == 'Green-winged Teal']

,true_alpha,common_name,scientific_name,t_and_e,comments,expected_alpha,display_name


2. The common_name was not changed

In [70]:
bbl_table[bbl_table['common_name'] == 'American Green-winged Teal']

,true_alpha,common_name,scientific_name,t_and_e,comments,expected_alpha,display_name


Now rerun the code above to make sure the above discrepancies were fixed.

We should only see discrepancies where the two authorities have the same alpha code for entirely different species or taxonomic groups, not ones where the common names are just a little different.

In [71]:
ibp_tc = ibp_table[['true_alpha', 'display_name', 'scientific_name', 'common_name']]
bbl_tc = bbl_table[['true_alpha', 'display_name', 'scientific_name', 'common_name']]

ibp_idxes_checked = []
for idx, row in ibp_tc.iterrows():
    ibp_idxes_checked.append(idx)
    bbl_for_this = bbl_tc[bbl_tc['true_alpha'] == row['true_alpha']]
    if len(bbl_for_this) > 0 and bbl_for_this['display_name'].values[0] != row['display_name']:
        print('IBP SPECIES:', ' -- '.join(row))
        print('BBL SPECIES:', ' -- '.join([str(x) for x in bbl_for_this.values[0]]))
        print()

IBP SPECIES: BARG -- Barnacle Goose -- Branta leucopsis -- Barnacle Goose
BBL SPECIES: BARG -- Bar-tailed Godwit -- Limosa lapponica -- Bar-tailed Godwit

IBP SPECIES: SCGO -- Small Canada Goose -- Branta canadensis parvipes -- Small Canada Goose
BBL SPECIES: SCGO -- Small Canada/Cackling Goose -- Branta canadensis parvipes -- Small Canada/Cackling Goose

IBP SPECIES: BLAG -- Black Guan -- Chamaepetes unicolor -- Black Guan
BBL SPECIES: BLAG -- Black-tailed Godwit -- Limosa limosa -- Black-tailed Godwit

IBP SPECIES: PAPI -- Passenger Pigeon -- Ectopistes migratorius -- Passenger Pigeon
BBL SPECIES: PAPI -- Pacific Pigeon -- Ducula pacifica -- Pacific Pigeon

IBP SPECIES: SLSH -- Slender Sheartail -- Doricha enicura -- Slender Sheartail
BBL SPECIES: SLSH -- San Clemente Loggerhead Shrike -- Lanius ludovicianus mearnsi -- San Clemente Loggerhead Shrike

IBP SPECIES: KEPE -- Kermadec Petrel -- Pterodroma neglecta -- Kermadec Petrel
BBL SPECIES: KEPE -- Kerguelen Petrel -- Aphrodroma brev

# Create a table with alpha codes and display names, to be searchable

Desired behavior if I type in...

The alpha code for a species with no conflicts and the same alpha codes: 
```
CODE: Species Name.
```

The conflicting alpha code for a species with a conflict, code not in use:
```
CODE: Not in use. 

Confusion species:
Species1 - IBPCODE - BBLCODE
Species2 - IBPCODE - BBLCODE
```

The conflicting alpha code for a species with a conflict, code in use by both for same species:

```
CODE: Species Name.

Confusion species:
Species1 - IBPCODE - BBLCODE
```


The conflicting alpha code for a species with a conflict, other scenario not described:

```
CODE: Conflict.

Confusion species:
Species1 - IBCO (IBP) - BBLCODE (BBL)
Species2 - IBPCODE - BBLCODE
```

The non-conflicting alpha code for a species with a conflict:

```
CODE: Species Name.

Confusion species:
Species1 - IBPCODE - BBLCODE
```

In [72]:
def code_in_use(code, row):
    if code in row['true_alpha'].values:
        return True
    else:
        return False

In [73]:
def expected_code_in_use(row):
    # Asserts that the expected code is unique
    [expected_code] = row['expected_alpha'].unique()

    return code_in_use(expected_code, row)



One (not quite foolproof) way to find out of the logic below is sound : return from this function at every print. run this function on every possible alpha code. make sure it always returns something.

Also, check the conflicts in the IBP document for good fodder for tests.

In [74]:
vc = ibp_table['expected_alpha'].value_counts()

In [75]:
vc[vc > 1]

expected_alpha
CAWR    5
COPO    4
SCAN    4
SBWO    4
BRWA    4
       ..
BTGW    2
BCTI    2
HADU    2
CIBE    2
GRFR    2
Name: count, Length: 208, dtype: int64

IN THE IBP TABLE: Are there any where the code used to resolve the conflict is the expected code of another species? 

In [78]:
expected_codes_ibp = ibp_table['expected_alpha'].unique()
for a in expected_codes_ibp:
    used = ibp_table[ibp_table['true_alpha'] == a]
    if used.shape[0] > 0:
        if used['expected_alpha'].values[0] != a:
            print('--'.join([str(x) for x in used.values[0]]))
        

nan--PROW--*--Prothonotary Warbler--Protonotaria citrea--PROCIT--nan--PRWA--Prothonotary Warbler
nan--MGWA--nan--MacGillivray's Warbler--Geothlypis tolmiei--GEOTOL--nan--MAWA--MacGillivray's Warbler


There are two! One is Prothonotary Warbler... which chafes against Puerto Rican Owl

In [79]:
ibp_table[ibp_table['expected_alpha'] == 'PROW']

,non_species,true_alpha,conflict,common_name,scientific_name,true_alpha_6,conflict_6,expected_alpha,display_name
911,NaN,PRIO,NaN,Puerto Rican Owl,Gymnasio nudipes,GYSNUD,*,PROW,Puerto Rican Owl


The other is MacGillivray's Warbler... which goes up against Middendorff's Grasshopper Warbler

In [80]:
ibp_table[ibp_table['expected_alpha'] == 'MGWA']

,non_species,true_alpha,conflict,common_name,scientific_name,true_alpha_6,conflict_6,expected_alpha,display_name
1545,NaN,MGRW,*,Middendorff's Grasshopper Warbler,Helopsaltes ochotensis,HELOCH,NaN,MGWA,Middendorff's Grasshopper Warbler


Let's check the same for the BBL table:

In [83]:
expected_codes_bbl = bbl_table['expected_alpha'].unique()
for a in expected_codes_bbl:
    used = bbl_table[bbl_table['true_alpha'] == a]
    if used.shape[0] > 0:
        if used['expected_alpha'].values[0] != a:
            print('--'.join([str(x) for x in used.values[0]]))
        

PROW--Prothonotary Warbler--Protonotaria citrea--Ontario--nan--PRWA--Prothonotary Warbler


The BBL apparently only has a problem with PROW, for the same reason though, but it resolves the conflict differently:

In [85]:
bbl_table[bbl_table['expected_alpha'] == 'PROW']

,true_alpha,common_name,scientific_name,t_and_e,comments,expected_alpha,display_name
477,PURO,Puerto Rican Owl,Gymnasio nudipes,NaN,Lock-on bands recommended. | Bagues enclenchan...,PROW,Puerto Rican Owl


Create some mini tables.

In [86]:
ibp_df = ibp_table[['true_alpha', 'display_name', 'expected_alpha']]
ibp_df.loc[:, 'authority'] = 'ibp'
bbl_df = bbl_table[['true_alpha', 'display_name', 'expected_alpha']]
bbl_df.loc[:, 'authority'] = 'bbl'

/var/folders/t6/c2h76xs16cn4jt3vqc68b5lm0000gr/T/ipykernel_21801/2404599857.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ibp_df.loc[:, 'authority'] = 'ibp'
/var/folders/t6/c2h76xs16cn4jt3vqc68b5lm0000gr/T/ipykernel_21801/2404599857.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbl_df.loc[:, 'authority'] = 'bbl'


In [87]:
def code_search(term, print_result=True, sep='<br>'):
    #sep="\n" is another possibility
    print_string = ""
    
    # Get all species that either have this expected alpha or use it as actual alpha
    term = term.upper()
    all_ibp = ibp_df[(ibp_table['expected_alpha'] == term) | (ibp_table['true_alpha'] == term)]
    all_bbl = bbl_df[(bbl_table['expected_alpha'] == term) | (bbl_table['true_alpha'] == term)]
    all_all = pd.concat([all_ibp, all_bbl])
    
    # Get all confusion species for anything with this expected alpha
    all_all = pd.concat([all_all, ibp_df[
        (ibp_df.expected_alpha.isin(all_all.expected_alpha))
        
    ]])
    all_all = pd.concat([all_all, bbl_df[
        (bbl_df.expected_alpha.isin(all_all.expected_alpha))
        
    ]])
    all_all = all_all.drop_duplicates()

    where_used = all_all[all_all.true_alpha == term]
    
    # Put IBP first
    where_used = where_used.sort_values('authority', ascending=True)
    
    ## STEP 1: PRINT WHETHER SOMEONE ACTUALLY USES THE CODE, AND IF SO, WHAT IT REFERS TO
    # If both use the code
    if where_used.shape[0] == 2:
        # e.g. YEWA
        # If both refer to the same species
        if len(where_used.display_name.unique()) == 1:
            print_string += f"{term}: {where_used.display_name.values[0]}{sep}"
            species = where_used.display_name.values[0]
    
        # If it differs between the two
        else:
            # e.g. KEPE
            print_string += (f"{term}: Code refers to different taxa in BBL and IBP.{sep}")
            print_string += f"{term}: {where_used.display_name.values[0]} ({where_used.authority.values[0].upper()}){sep}"
            print_string += f"{term}: {where_used.display_name.values[1]} ({where_used.authority.values[1].upper()}){sep}"
            species = where_used.display_name.values[0], where_used.display_name.values[1]
    
    # If only one authority uses the code for one species
    elif where_used.shape[0] == 1:
        different_code = all_all[
            (all_all.display_name == where_used.display_name.values[0]) &
            (all_all.true_alpha != term)
        ]
        
        # If that species does not appear in the other authority under a different code
        # e.g. WTPT
        if different_code.shape[0] == 0:
            print_string += f"{term}: {where_used.display_name.values[0]} ({where_used.authority.values[0].upper()}){sep}"
            species = where_used.display_name.values[0]
        
        # If that species does appear in the other authority under a different code
        # e.g. CANG
        else:
            print_string += f"{term}: {where_used.display_name.values[0]} ({different_code.authority.values[0].upper()}: {different_code.true_alpha.values[0].upper()}){sep}"
            species = where_used.display_name.values[0]
            #print_string += f"{term}: {where_used.display_name.values[0]} ({where_used.authority.values[0].upper()}){sep}"
            #print_string += f"[{different_code.true_alpha.values[0]}: {where_used.display_name.values[0]} ({different_code.authority.values[0].upper()})]{sep}"
            #print_string += f"({different_code.authority.values[0].upper()} code for {where_used.display_name.values[0]}: {different_code.true_alpha.values[0]}){sep}"
            
    # If neither use the code
    # E.g. PRWA
    else:
        assert where_used.shape[0] == 0
        if all_all.shape[0] > 0:
            print_string += f"{term}: Code not in use.{sep}"
            species = ''
        else:
            print_string += f"{term}: Code not found.{sep}"
            return False, False
        
    ## STEP 2: PRINT CONFUSION SPECIES.
    confusions = all_all[
        (all_all.true_alpha != term) &
        (~all_all.display_name.isin(where_used.display_name.values))
    ]
    if confusions.shape[0] > 0:
        print_string += f"{sep}Confusion species:{sep}"
    for confusion_species in confusions.display_name.unique():
        instances = confusions[confusions.display_name == confusion_species]
        assert instances.shape[0] in [1, 2]
        
        # If the confusion species appears in both authorities
        if instances.shape[0] == 2:
            
            # If the authorities have different codes for the confusion species
            # E.g. LESP --> different codes for Lesser Sand-Plover and Least Storm-Petrel
            if len(instances.true_alpha.unique()) == 2:
            #    print_string += f"{instances.true_alpha.values[0]}: {confusion_species} ({instances.authority.values[0].upper()}){sep}"
            #    print_string += f"{instances.true_alpha.values[1]}: {confusion_species} ({instances.authority.values[1].upper()}){sep}"
                print_string += f"{instances.true_alpha.values[0]}/{instances.true_alpha.values[1]}: {confusion_species}{sep}"

            # If the authorities have the same code for the confusion species
            # E.g. MAWA --> MacGillivray's Warbler uses MGWA for both authorities
            else:
                print_string += f"{instances.true_alpha.values[0]}: {confusion_species}{sep}"
        
        # If the confusion species appears in only one authority
        # E.g. MAWA --> Mangrove Warbler appears only in IBP
        else:
            print_string += f"{instances.true_alpha.values[0]}: {confusion_species} ({instances.authority.values[0].upper()}){sep}"
    
    #if print_result:
    #    print(print_string) 
        
    return print_string, term, species
code_search('yewa')

('YEWA: Yellow Warbler<br>', 'YEWA', 'Yellow Warbler')

Let's check some of these trickier codes to make sure this is working as expected.

In [91]:
print(code_search("LCTH"))
print(code_search("leth"))
print(code_search("KEPE"))
print(code_search("SBOH"))
print(code_search("AGWT"))
print(code_search("gwte"))
print(code_search("grej"))
print(code_search("cang"))
print(code_search("hard"))
print(code_search("lesp"))
print(code_search("prow"))
print(code_search("scgo"))
print(code_search("mgwa"))


("LCTH: LeConte's Thrasher<br>", 'LCTH', "LeConte's Thrasher")
("LETH: Code not in use.<br><br>Confusion species:<br>LCTH: LeConte's Thrasher<br>", 'LETH', '')
('KEPE: Code refers to different taxa in BBL and IBP.<br>KEPE: Kerguelen Petrel (BBL)<br>KEPE: Kermadec Petrel (IBP)<br>', 'KEPE', ('Kerguelen Petrel', 'Kermadec Petrel'))
('SBOH: Spotted x Barred Owl Hybrid<br>', 'SBOH', 'Spotted x Barred Owl Hybrid')
('AGWT: American Green-winged Teal<br><br>Confusion species:<br>GWTE: Green-winged Teal (IBP)<br>', 'AGWT', 'American Green-winged Teal')
('GWTE: Green-winged Teal (IBP)<br><br>Confusion species:<br>AGWT: American Green-winged Teal (BBL)<br>', 'GWTE', 'Green-winged Teal')
('GREJ: Great Jacamar (IBP)<br><br>Confusion species:<br>GRJA: Green Jay<br>', 'GREJ', 'Great Jacamar')
('CANG: Canada Goose (BBL: CAGO)<br><br>Confusion species:<br>CACG: Cackling Goose (IBP)<br>', 'CANG', 'Canada Goose')
('HARD: Harlequin Duck (IBP: HADU)<br><br>Confusion species:<br>HAWD: Hawaiian Duck<br>', '

In [95]:

all_codes_appearing = []
all_codes_appearing.extend(ibp_table.true_alpha.unique())
all_codes_appearing.extend(ibp_table.expected_alpha.unique())
all_codes_appearing.extend(bbl_table.true_alpha.unique())
all_codes_appearing.extend(bbl_table.expected_alpha.unique())
all_codes = sorted(set(all_codes_appearing))

print_column = []
code_column = []
species_column = []
for code in all_codes:
    ret = code_search(code, print_result=False)
    print_column.append(ret[0])
    code_column.append(ret[1])
    species_column.append(ret[2])



In [96]:
df = pd.DataFrame({"Result":print_column, "code":code_column, "species":species_column})
df.head()

,Result,code,species
0,ABDD: Code not in use.<br><br>Confusion specie...,ABDD,
1,ABDU: American Black Duck<br>,ABDU,American Black Duck
2,ABDX: American Black Duck Dominant x Mallard H...,ABDX,American Black Duck Dominant x Mallard Hybrid
3,ABFL: Asian Brown Flycatcher (IBP)<br>,ABFL,Asian Brown Flycatcher
4,ABOW: American Barn Owl<br>,ABOW,American Barn Owl


# Now make a searchable HTML table.

### Remove hybrids and intergrades

Hybrids:

In [97]:
df = df[~df['Result'].str.lower().str.contains("hybrid", na=False)]

Intergrades:

In [98]:
df = df[~df['Result'].str.lower().str.contains("intergrade", na=False)]


### Put search terms in a single column

In [99]:
df

,Result,code,species
1,ABDU: American Black Duck<br>,ABDU,American Black Duck
3,ABFL: Asian Brown Flycatcher (IBP)<br>,ABFL,Asian Brown Flycatcher
4,ABOW: American Barn Owl<br>,ABOW,American Barn Owl
5,ABTO: Abert's Towhee<br>,ABTO,Abert's Towhee
6,ACCA: Audubon's Crested Caracara<br>,ACCA,Audubon's Crested Caracara
...,...,...,...
2664,ZEBD: Zebra Dove<br><br>Confusion species:<br>...,ZEBD,Zebra Dove
2665,ZEDO: Code not in use.<br><br>Confusion specie...,ZEDO,
2666,ZEND: Zenaida Dove<br><br>Confusion species:<b...,ZEND,Zenaida Dove
2667,ZIPE: Zino's Petrel (IBP)<br>,ZIPE,Zino's Petrel


In [100]:
df['search'] = df['code'].astype(str) + ' ' + df['species'].astype(str)

In [101]:
df.head()

,Result,code,species,search
1,ABDU: American Black Duck<br>,ABDU,American Black Duck,ABDU American Black Duck
3,ABFL: Asian Brown Flycatcher (IBP)<br>,ABFL,Asian Brown Flycatcher,ABFL Asian Brown Flycatcher
4,ABOW: American Barn Owl<br>,ABOW,American Barn Owl,ABOW American Barn Owl
5,ABTO: Abert's Towhee<br>,ABTO,Abert's Towhee,ABTO Abert's Towhee
6,ACCA: Audubon's Crested Caracara<br>,ACCA,Audubon's Crested Caracara,ACCA Audubon's Crested Caracara


# Write as json variable in HTML file

Make the lookup table available as a variable, `search_data`, which will be loaded as a script in `popup.html` 

Need to save this in both the website (`../docs/`) and extension (`../extension/`) directories:

In [103]:
with open("../docs/resources/alpha_data.html", "w+") as f:
    j = df[['search', 'Result']].set_index("search").transpose().to_json()
    f.write(f"var search_data = {j};")

with open("../extension/resources/alpha_data.html", "w+") as f:
    j = df[['search', 'Result']].set_index("search").transpose().to_json()
    f.write(f"var search_data = {j};")

